<a href="https://colab.research.google.com/github/sipocz/Blood/blob/main/Blood_ldl_model_20211215_ipynb_m%C3%A1solata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install lazypredict

In [2]:
__PROJECT_SOURCE__="COLAB"

In [3]:
if __PROJECT_SOURCE__=="COLAB":
    # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    from google.colab import drive
    drive.mount('/content/drive')
    fname_dir="/content/blood/"
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv"
    fname=fname_url.split("/")[-1]
elif __PROJECT_SOURCE__=="LOCAL":
    fname_dir="sdfsdfsd" #working dir
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv" #data source dir
    


Mounted at /content/drive


In [4]:
!mkdir $fname_dir

In [5]:
!cp $fname_url $fname_dir

In [6]:
if __PROJECT_SOURCE__=="COLAB":
    drive.flush_and_unmount()
    print('Unmount Google Drive :-(')

Unmount Google Drive :-(


## Feldolgozás

In [7]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [8]:
import pandas as pd

In [9]:
fnames_list=["ldl1_agg.csv","ldl2_agg.csv"]

In [10]:
file_name1=fname_dir+fnames_list[0]
print(file_name1)

/content/blood/ldl1_agg.csv


In [11]:
file_name2=fname_dir+fnames_list[1]
print(file_name2)

/content/blood/ldl2_agg.csv


In [12]:
df_agg1= pd.read_csv(file_name1)
df_agg1.describe()
df_agg1.drop(df_agg1[df_agg1.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [13]:
df_agg1.head()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
0,0,0.535147,0.537960,0.541943,0.548029,0.553548,0.560042,0.566466,0.574085,0.586266,0.599614,0.615089,0.626659,0.633565,0.637011,0.639238,0.638972,0.637331,0.635508,0.632157,0.628089,0.624585,0.621561,0.617845,0.614378,0.610104,0.607131,0.604818,0.602482,0.601290,0.599995,0.598909,0.599053,0.599842,0.601187,0.602825,0.606265,0.609677,0.614040,0.618327,...,1.688604,1.674469,1.660716,1.648262,1.630642,1.618590,1.606035,1.591950,1.580556,1.565986,1.555014,1.542866,1.533852,1.525125,1.514723,1.503877,1.495593,1.488977,1.479454,1.472006,1.465699,1.460134,1.458216,1.456837,1.456203,1.459403,1.460975,1.457055,1.457455,1.454225,1.447507,1.450268,1.451860,1.461692,1.471279,2440,39.057167,46.649500,105.6,1
1,1,0.538019,0.538283,0.542379,0.545535,0.549656,0.555612,0.563389,0.570042,0.579115,0.590913,0.604809,0.620944,0.628157,0.630772,0.631071,0.630588,0.628943,0.627554,0.625168,0.623034,0.620265,0.617370,0.614614,0.611324,0.608916,0.606037,0.603702,0.602188,0.600821,0.599539,0.598978,0.599364,0.599753,0.601123,0.603251,0.605764,0.608760,0.612281,0.615855,...,1.564822,1.550711,1.534420,1.521360,1.509736,1.496078,1.484581,1.472423,1.458472,1.450467,1.436744,1.426040,1.417529,1.408495,1.397023,1.391276,1.383204,1.374296,1.368845,1.361276,1.354829,1.349905,1.344804,1.343582,1.340867,1.341139,1.342966,1.342546,1.342661,1.343233,1.339520,1.331402,1.329552,1.334918,1.308238,2476,38.457833,51.680833,72.4,1
2,2,0.480685,0.484010,0.489706,0.494084,0.500947,0.507894,0.513360,0.520308,0.530952,0.543218,0.557018,0.567722,0.574330,0.577556,0.578793,0.578903,0.577392,0.575224,0.572716,0.568465,0.565585,0.562637,0.559241,0.556354,0.552418,0.549454,0.547636,0.545892,0.544410,0.543401,0.542475,0.543075,0.543687,0.544753,0.546528,0.549802,0.553025,0.556793,0.560706,...,1.539243,1.526498,1.512998,1.500887,1.486508,1.474466,1.461664,1.452243,1.439495,1.427213,1.416068,1.406622,1.398762,1.389438,1.379374,1.372073,1.363920,1.357083,1.351891,1.343235,1.339888,1.334037,1.332106,1.329624,1.332323,1.330829,1.334422,1.328862,1.324434,1.315813,1.300146,1.295544,1.293936,1.296249,1.294994,2478,42.028167,43.518667,68.4,1
3,3,0.505587,0.510463,0.515027,0.520545,0.528644,0.539653,0.547931,0.553654,0.560278,0.571240,0.582939,0.597353,0.603132,0.606054,0.607849,0.608324,0.607437,0.605870,0.602386,0.599726,0.596480,0.593364,0.590350,0.587280,0.584094,0.580810,0.578204,0.576421,0.574798,0.572748,0.572369,0.572139,0.572084,0.572998,0.574243,0.576048,0.578608,0.582545,0.586093,...,1.453705,1.442506,1.431383,1.420372,1.409361,1.398739,1.384815,1.375346,1.364956,1.358352,1.346600,1.337193,1.329186,1.319507,1.312434,1.305864,1.297765,1.292396,1.287311,1.279727,1.274297,1.272475,1.267522,1.265180,1.264382,1.263661,1.261225,1.255700,1.249614,1.239259,1.224037,1.213512,1.204171,1.216543,1.208677,2559,43.543333,23.219333,106.0,1
4,4,0.546671,0.548270,0.551964,0.555424,0.560574,0.566887,0.572948,0.580261,0.591718,0.605186,

In [14]:
df_agg2= pd.read_csv(file_name2)
df_agg2.describe()
df_agg2.drop(df_agg2[df_agg2.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [15]:
df_agg=pd.concat([df_agg1,df_agg2], ignore_index=True)

In [16]:
df_agg.tail()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
537,405,0.559508,0.557981,0.562073,0.566272,0.571244,0.577397,0.583213,0.590163,0.598694,0.610475,0.623472,0.636641,0.642579,0.645626,0.647103,0.645668,0.643687,0.640753,0.637975,0.635578,0.633089,0.630470,0.627600,0.624106,0.622028,0.619228,0.617372,0.616006,0.614953,0.614089,0.614401,0.614494,0.615416,0.617037,0.619308,0.623084,0.626258,0.629867,0.634516,...,1.440180,1.429945,1.421697,1.412292,1.403772,1.395936,1.385628,1.377189,1.370285,1.363224,1.354198,1.347067,1.339531,1.332175,1.326958,1.320780,1.315768,1.311262,1.304993,1.299778,1.296999,1.292156,1.289398,1.285569,1.286972,1.283523,1.281386,1.279686,1.274868,1.271164,1.265842,1.263005,1.266337,1.297181,1.266201,11886,36.536500,65.203000,88.54285,1
538,406,0.587427,0.585250,0.589764,0.594061,0.599806,0.604902,0.609408,0.614733,0.620852,0.630668,0.641564,0.653106,0.657908,0.659866,0.660317,0.658655,0.655788,0.652237,0.648927,0.646154,0.643198,0.640632,0.637537,0.633689,0.631356,0.628085,0.625785,0.624119,0.622667,0.621206,0.620691,0.620383,0.620737,0.621599,0.623025,0.625937,0.628400,0.631338,0.635659,...,1.463203,1.454463,1.445494,1.436059,1.426811,1.417521,1.406092,1.397415,1.388936,1.383776,1.372381,1.364249,1.355797,1.347775,1.341057,1.333965,1.328754,1.321832,1.315572,1.309635,1.303772,1.299180,1.295138,1.292522,1.289935,1.287213,1.282831,1.277070,1.270260,1.253691,1.242504,1.236326,1.236166,1.250518,1.236380,11889,39.054667,56.012500,159.68645,3
539,407,0.477241,0.481792,0.483792,0.489861,0.497037,0.504735,0.509637,0.514202,0.520144,0.530000,0.541345,0.550381,0.555803,0.558563,0.559343,0.558502,0.556676,0.554130,0.551494,0.547095,0.543851,0.540543,0.537108,0.533737,0.530775,0.527208,0.524530,0.523219,0.521271,0.520013,0.519327,0.518718,0.518963,0.519878,0.520868,0.522727,0.525264,0.528919,0.532656,...,1.494378,1.478303,1.463623,1.448563,1.434307,1.419192,1.402043,1.388613,1.375551,1.366222,1.351601,1.340437,1.327134,1.318319,1.309765,1.300136,1.292696,1.283508,1.275061,1.269340,1.263197,1.258333,1.253829,1.251677,1.249435,1.246182,1.243464,1.237783,1.227517,1.206936,1.179302,1.162252,1.156460,1.167845,1.159627,11890,46.116000,46.849000,99.20000,1
540,408,0.565670,0.564820,0.569749,0.573770,0.580790,0.586531,0.591289,0.596792,0.603492,0.613606,0.625469,0.637730,0.642756,0.645300,0.646095,0.644386,0.641853,0.638425,0.635619,0.633162,0.630523,0.627955,0.624878,0.621162,0.618793,0.615519,0.613485,0.611918,0.610617,0.609168,0.608868,0.608688,0.609372,0.610378,0.612109,0.615408,0.618294,0.621513,0.626012,...,1.506006,1.496877,1.486883,1.475934,1.466819,1.456462,1.445260,1.435506,1.426113,1.419729,1.409409,1.402097,1.391310,1.384826,1.377594,1.371748,1.366276,1.358685,1.354177,1.348492,1.342599,1.337098,1.332178,1.329692,1.329856,1.327782,1.323786,1.317777,1.305306,1.290804,1.276198,1.268350,1.263184,1.284706,1.269695,11891,40.384833,51.917167,164.71290,3
541,409,0.462006,0.463919,0.466006,0.470398,0.474160,0.479

In [17]:
df_agg.describe()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
count,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,...,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000
mean,170.702952,0.520451,0.522427,0.525972,0.530680,0.536854,0.543551,0.549648,0.555232,0.563119,0.574407,0.587313,0.598591,0.605160,0.608182,0.609431,0.609150,0.607543,0.605090,0.602129,0.598775,0.595773,0.592707,0.589487,0.585683,0.582415,0.579220,0.576781,0.574689,0.572943,0.571480,0.570644,0.570247,0.570480,0.571400,0.572933,0.575169,0.577869,0.581283,0.585061,...,1.497814,1.485553,1.474109,1.463112,1.450626,1.439371,1.428135,1.417230,1.406798,1.398556,1.385953,1.377019,1.368533,1.360171,1.351819,1.344645,1.337181,1.330049,1.323455,1.316835,1.311906,1.307739,1.304195,1.301982,1.301178,1.300802,1.300394,1.299197,1.295539,1.289453,1.282270,1.275940,1.272953,1.287608,1.275556,6927.533210,42.111418,39.222955,111.436592,1.485240
std,120.507914,0.040351,0.039984,0.040093,0.040035,0.040144,0.040521,0.040435,0.040376,0.040171,0.039981,0.039891,0.039888,0.039833,0.039825,0.039900,0.039866,0.039767,0.039718,0.039647,0.039748,0.039703,0.039702,0.039726,0.039846,0.040008,0.040042,0.040114,0.040156,0.040191,0.040191,0.040187,0.040152,0.040178,0.040161,0.040140,0.040145,0.040169,0.040266,0.040271,...,0.091700,0.090305,0.088924,0.087603,0.085971,0.084668,0.083410,0.082486,0.081525,0.080697,0.079354,0.078591,0.077906,0.077287,0.076565,0.076015,0.075402,0.074832,0.074297,0.073843,0.073528,0.073310,0.073180,0.073283,0.073591,0.074264,0.075230,0.076718,0.079323,0.083310,0.088549,0.093928,0.098151,0.104133,0.101605,3094.955454,3.420590,8.948404,44.306810,1.031287
min,0.000000,0.400606,0.403625,0.407266,0.411518,0.416126,0.421518,0.426629,0.433139,0.444069,0.456182,0.469249,0.480239,0.486473,0.489661,0.490777,0.490226,0.488814,0.486476,0.483668,0.479653,0.476146,0.473155,0.469834,0.466424,0.462388,0.459335,0.457297,0.455538,0.453853,0.452486,0.451719,0.451624,0.452026,0.453133,0.454851,0.457279,0.460600,0.463820,0.467916,...,1.202957,1.193580,1.184336,1.175013,1.163368,1.154944,1.145616,1.137301,1.128859,1.118666,1.110864,1.102883,1.096296,1.089810,1.080864,1.075002,1.068992,1.063488,1.057937,1.052125,1.047685,1

In [18]:

import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [19]:
X_columns=df_agg.columns[1:-5]
y_columns=df_agg.columns[-1] # the category is at the end of columns
X_=df_agg[X_columns]
y_=df_agg[y_columns]

In [20]:
X_columns

Index(['absorbance0', 'absorbance1', 'absorbance2', 'absorbance3',
       'absorbance4', 'absorbance5', 'absorbance6', 'absorbance7',
       'absorbance8', 'absorbance9',
       ...
       'absorbance160', 'absorbance161', 'absorbance162', 'absorbance163',
       'absorbance164', 'absorbance165', 'absorbance166', 'absorbance167',
       'absorbance168', 'absorbance169'],
      dtype='object', length=170)

In [21]:
y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 

In [22]:
#y_.replace(2,0, inplace=True)

In [23]:
y_=np.array(y_)

In [24]:
y_=y_.reshape(-1,1)

In [25]:
y_columns

'cholesterol_ldl_human'

In [79]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

X_columns=df_agg.columns[1:-5]
y_columns=df_agg.columns[-1] # the category is at the end of columns
X_=df_agg[X_columns]
y_=df_agg[y_columns]

y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 

y_=np.array(y_)
y_=y_.reshape(-1,1)



__X_SCALER__="normal"

__DNN_MODE__="regression"

__Y_SCALER__="minmax"

if __DNN_MODE__=="regression":
    y_columns=df_agg.columns[-2] # the category is at the  -2  column
    y_=df_agg[y_columns]
    y_=np.array(y_)
    y_=y_.reshape(-1,1)


elif __DNN_MODE__=="classification":
    y_columns=df_agg.columns[-1] # the category is at the end of columns
    y_=df_agg[y_columns]
    y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 
    y_=np.array(y_)
    y_=y_.reshape(-1,1)

y=y_
print(y[0:10])

if __X_SCALER__=="normal":
    X_normalizer = StandardScaler() 
    X=X_normalizer.fit_transform(X_)

if __Y_SCALER__=="normal":
    y_normalizer = StandardScaler() 
    y=y_normalizer.fit_transform(y_,)
    print(f"Itt: {y}")

if __X_SCALER__=="minmax":
    X_minmax = MinMaxScaler() 
    X=X_minmax.fit_transform(X_)

if __Y_SCALER__=="minmax":
    y_minmax = MinMaxScaler() 
    y=y_minmax.fit_transform(y_)
    




[[105.6]
 [ 72.4]
 [ 68.4]
 [106. ]
 [ 31.2]
 [ 40.6]
 [ 82.6]
 [ 32.7]
 [ 64.2]
 [151. ]]


In [80]:
y_normalizer.get_params()

{'copy': True, 'with_mean': True, 'with_std': True}

In [81]:
y

array([[0.32831424],
       [0.21963993],
       [0.20654664],
       [0.32962357],
       [0.08477905],
       [0.11554828],
       [0.25302782],
       [0.08968903],
       [0.19279869],
       [0.47692308],
       [0.29296236],
       [0.23993453],
       [0.34533552],
       [0.46775777],
       [0.25302782],
       [0.40883797],
       [0.67528642],
       [0.15351882],
       [0.2811784 ],
       [0.10572831],
       [0.3806874 ],
       [0.35711948],
       [0.35777414],
       [0.3695581 ],
       [0.2386252 ],
       [0.31914894],
       [0.59934534],
       [0.43371522],
       [0.4212766 ],
       [0.52405892],
       [0.29427169],
       [0.20589198],
       [0.498527  ],
       [0.498527  ],
       [0.22487725],
       [0.24189853],
       [0.46513912],
       [0.37217676],
       [0.48281506],
       [0.39849427],
       [0.30343699],
       [0.12981997],
       [0.1894599 ],
       [0.33165303],
       [0.26278232],
       [0.34297872],
       [0.26599018],
       [0.282

In [82]:
'''
import wandb
from wandb.keras import WandbCallback

wandb.init(config={"hyper": "parameter"})
'''



'\nimport wandb\nfrom wandb.keras import WandbCallback\n\nwandb.init(config={"hyper": "parameter"})\n'

In [83]:
def blood_model_2():
    #clear_session()

    keras.backend.clear_session()

    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,))

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="HeNormal"
    d1=Dense(1000,activation="sigmoid",kernel_initializer=initializer)(input1)
    d1=Dense(100,activation="sigmoid",kernel_initializer=initializer)(d1)
    #d1=Dense(100,activation="selu",kernel_initializer=initializer)(d1)


    #softmax
    pred=Dense(1,activation="sigmoid",)(d1)

    
    
    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adamax(learning_rate=0.001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="CategoricalCrossentropy",
        optimizer=opt,
        metrics=["Accuracy"])
    return(model)

In [84]:
!pip install wandb
!wandb login

wandb: Currently logged in as: sipoczlaszlo (use `wandb login --relogin` to force relogin)


In [85]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="BM_Norm_3", entity="sipoczlaszlo")

MSE,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_MSE,▃▁▂▁▂▂▁▁▂▁▁▃▁█▁▂▃▁▂▃▂▁▂▃▂▂▂▄▃▃▃▄▃▃▂▃▃▂▃▃
val_loss,▃▁▂▁▂▂▁▁▂▁▁▃▁█▁▂▃▁▂▃▂▁▂▃▂▂▂▄▃▃▃▄▃▃▂▃▃▂▃▃
MSE,0.87777
best_epoch,20
best_val_loss,1.13807
epoch,48
loss,0.87777


In [86]:
def blood_model_regression():
    #clear_session()

    keras.backend.clear_session()

    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.16 
    drop_frac1=0.0



    input1=Input(shape=(input_len,))

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="HeNormal"
    initializer2="GlorotNormal"
    
    d1=Dense(5512,activation="relu",kernel_initializer=initializer)(input1)
    d1=Dropout(drop_frac0)(d1)
    
    d1=Dense(256,activation="relu",kernel_initializer=initializer)(d1)
    #d1=Dropout(drop_frac1)(d1)
    #d1=Dense(128,activation="tanh",kernel_initializer=initializer2)(d1)
    # d1=Dropout(drop_frac1)(d1) 

    #softmax
    pred=Dense(1, kernel_initializer=initializer)(d1)

    
    
    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.SGD(learning_rate=0.0000001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="MSE",
        optimizer=opt,
        metrics=["MSE"])
    return(model)

In [87]:
__DNN_MODE__

'regression'

In [88]:
if __DNN_MODE__=="classification":
    model_name="classification_"
    def scheduler(epoch, lr):
        return 0.001

    callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

    callbacks = [callback_LR,
            
            #savemodela,
            ModelCheckpoint(filepath=model_name+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                            verbose=1, save_best_only=True, mode='min')]



In [89]:
if __DNN_MODE__=="regression":
    model_name="regression_"
    def scheduler(epoch, lr):
        if epoch<50:
            return 0.001
        elif epoch <100:
            return 0.0005
        elif epoch <150:
            return 0.0001
        return 0.00005    


    callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

    callbacks = [callback_LR,
            
            #savemodela,
            ModelCheckpoint(filepath=model_name+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                            verbose=1, save_best_only=True, mode='min')]



In [90]:
if __DNN_MODE__=="classification":
    new_model_2=blood_model_2()

In [91]:
if __DNN_MODE__=="regression":
    new_model_2=blood_model_regression()

170


In [92]:
!rm *.hdf5

In [93]:
X

array([[ 0.36453647,  0.38885105,  0.39872672, ...,  1.82445842,
         1.673298  ,  1.92809129],
       [ 0.43579244,  0.39694511,  0.40961367, ...,  0.57719086,
         0.45474503,  0.32195174],
       [-0.9863988 , -0.96168426, -0.90536406, ...,  0.21397963,
         0.08306335,  0.19148561],
       ...,
       [-1.07183495, -1.01721066, -1.05299893, ..., -1.18797608,
        -1.15115283, -1.14202762],
       [ 1.12166936,  1.0612296 ,  1.09288581, ..., -0.09962229,
        -0.02788948, -0.05773341],
       [-1.44973193, -1.46461933, -1.4970304 , ..., -1.13059767,
        -1.05839207, -1.05836683]])

In [ ]:
for i in range(5):
    if __DNN_MODE__=="regression":
        new_model_2.summary()
        history=new_model_2.fit(X,
          y,
          epochs=200, 
          batch_size=3,
          validation_split=0.1,
          verbose=1,
          callbacks=[callbacks,WandbCallback()],
          shuffle=True
          
          )


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 170)]             0         
                                                                 
 dense (Dense)               (None, 5512)              942552    
                                                                 
 dropout (Dropout)           (None, 5512)              0         
                                                                 
 dense_1 (Dense)             (None, 256)               1411328   
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2,354,137
Trainable params: 2,354,137
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
156/163 [===========================>..

In [ ]:
---

In [ ]:
#new_model_2.load_weights("./_col_0.00098_0.00173_.xhdf5")

In [ ]:
new_model_2.load_weights("./regression__0.04629_0.03222_.hdf5")

In [ ]:
y_pred=new_model_2.predict(X)
result_df=pd.DataFrame()
y_pred_col=[i[0] for i in y_pred]
y_col=[i[0] for i in y]

result_df["y"]=y_col
result_df["y_pred"]=y_pred_col

In [ ]:
import plotly.express as px

fig = px.scatter(result_df, x="y", y="y_pred",width=600, height=400 )
fig.show()

In [ ]:
---

In [ ]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score,mean_absolute_error


clf = RandomForestRegressor(max_depth=70, random_state=0,n_estimators=300)
clf.fit(X_train, y_train)

randomforest_predict=clf.predict(X_train)


In [ ]:
print(f"Accuracy: {mean_absolute_error(y_train,randomforest_predict)}")

In [ ]:
randomforest_predict=clf.predict(X_test)
print(f"Accuracy: {mean_absolute_error(y_test,randomforest_predict)}")

In [ ]:
---

In [ ]:
y

In [ ]:
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X[:-100], X[-100:], y[:-100], y[-100:])

In [ ]:
print(models)

In [ ]:
yo=[i[0] for i in y_]
yo

In [ ]:
X

In [ ]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose=2, ignore_warnings=False, custom_metric=None)
models,predictions = clf.fit(X[:-100], X[-100:], y[:-100], y[-100:])

In [ ]:
print(models)

In [ ]:
x_counter=1

In [ ]:
X_train=X[:-1*x_counter]
X_test=X[-1*x_counter:]
y_train=yo[:-1*x_counter]
y_test =yo[-1*x_counter:]
